In [68]:
import numpy as np
print('Hello')



Hello


In [3]:
# -*- coding: utf-8 -*-
"""
Created on Fri Apr  6 16:09:25 2018

@author: Shubadra
"""

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import numpy as np
import string
import re
from functools import reduce
from nltk.stem import PorterStemmer
from collections import Counter 
from nltk.corpus import wordnet 
from nltk.stem import WordNetLemmatizer 


# def get_pos( word ):
#     w_synsets = wordnet.synsets(word)

#     pos_counts = Counter()
#     pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
#     pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
#     pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
#     pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
#     most_common_pos_list = pos_counts.most_common(3)
#     return most_common_pos_list[0][0]

find = lambda searchList, elem: [[i for i, x in enumerate(searchList) if x == e] for e in elem]
fname = 'project_2_train/' + 'data 2_train.csv'

stopwords_set = set(stopwords.words('english'))
stopwords_set = {'i', 'shan', 'just', 'how', 'each', 'out', 'themselves', 'their', 'before', 'were', 'very', 'as', 'further', 'his', 'a', 'once', 'youve', 'y', 'is', 'shouldve', 'youll', 'on', 'd', 'm', 'under', 'haven', 'which', 'only', 'them', 'was', 'by', 'needn', 'whom', 'that', 'when', 's', 'isn', 'its', 'no', 'wasn', 'in', 'we', 'theirs', 'those', 'this', 'having', 'and', 'ain', 'most', 'up', 'off', 'being', 'aren', 'shouldn', 'ourselves', 'from', 'down', 'herself', 'her', 'you', 'are', 'its', 'who', 'the', 'here', 'where', 'your', 'youd', 'she', 'didn', 'weren', 'about', 'has', 'our', 'an', 'yourselves', 'or', 'hasn', 'again', 'while', 'does', 'him', 'shes', 'above', 'below', 'itself', 'to', 'through', 'will', 'couldn', 'hers', 'they', 'doing', 'because', 'he', 'what', 'such', 'youre', 'nor', 'too', 'should', 'ours', 'then', 'himself', 'all', 'of', 'mightn', 'between', 'now', 'against', 'some', 'with', 'until', 'am', 'other', 'at', 'can', 'over', 'mustn', 'wouldn', 'do', 'for', 'after', 'hadn', 'me', 'been', 'same', 'doesn', 'my', 'these', 'll', 'did', 'had', 'it', 'so', 'ma', 'during', 'than', 'o', 'yourself', 'own', 'have', 're', 've', 'be', 'why', 't', 'there', 'more', 'won', 'yours', 'few', 'into', 'thatll', 'any', 'myself', 'both', 'don', 'if'}
# print(stopwords_set)
# stopwords_set.remove('but')
# stopwords_set.remove('not')

sentence_weights = []
X, Y, X1 = [], [], []

f = open(fname, 'r')
for i, line in enumerate(f):
    if i != 0:
        ''' Splitting the columns based on comma - since it is csv'''
        columns = line.split(',')

        ''' The comma in the actual sentence was represented as [commma] because of csv format, replace that '''
        columns[1] = columns[1].replace('[comma]', ',')
        columns[2] = columns[2].replace('[comma]', ',')

        ''' Some aspect terms are mic in the sentence where computer/mic is present '''
        columns[1] = columns[1].replace('-', ' ')
        columns[2] = columns[2].replace('-', ' ')
        columns[1] = columns[1].replace('/', ' ')
        columns[2] = columns[2].replace('/', ' ')

        ''' NOt used anymore'''
        tokenizer = RegexpTokenizer(r'\w+') # doesn't work if the special char is in the token
        # columns[2] = tokenizer.tokenize(columns[2].lower())

        '''Tokenize the words'''
        columns[1] = word_tokenize(columns[1])
        columns[2] = word_tokenize(columns[2])

        for j, elem in enumerate(columns[1]):
            '''Remove special characters'''
            columns[1][j] = re.sub('[^0-9a-zA-Z]+', '', elem).lower()
            ''' didn't is tokenized as did n't hence following code '''
            if columns[1][j] == 'nt':
                columns[1][j] = 'not'

        for j, elem in enumerate(columns[2]):
            '''Remove special characters'''
            columns[2][j] = re.sub('[^0-9a-zA-Z]+', '', elem).lower()
            ''' didn't is tokenized as did n't hence following code '''
            if columns[2][j] == 'nt':
                columns[2][j] = 'not'

        '''Remove empty string tokens'''
        columns[1] = [x.strip() for x in columns[1] if x.strip() != '']
        columns[2] = [x.strip() for x in columns[2] if x.strip() != '']

        ''' Remove stop words '''
        columns[1] = [word for word in columns[1] if word not in stopwords_set]
        columns[1] = [word for word in columns[1] if word not in stopwords_set]
        columns[2] = [word for word in columns[2] if word not in stopwords_set]
        
#         '''Lemmatizing followed by Stemming'''
#         ps = PorterStemmer()
#         wnl = WordNetLemmatizer()
#         for word_index, word in enumerate(columns[1]):
#             columns[1][word_index] = ps.stem(word)
#         for aspect_term_index, aspect_term in enumerate(columns[2]):
#             columns[1][aspect_term_index] = ps.stem(wnl.lemmatize(aspect_term))
 
        columns[1] = ' '.join(columns[1])
        columns[2] = ' '.join(columns[2])

        ''' The aspect term location given is not proper - hence extracting the location by ourselves '''
        columns[3] = []
        ''' Finds the location of the aspect term in the sentence '''
        for m in re.finditer(columns[2], columns[1]):
            columns[3].append([m.start(), m.end()])
            ''' since some aspect terms are not surrounded by spaces, hence below code
            NOT USED ANYMORE
            '''
            if m.start() != 0 and columns[1][m.start() - 1] != ' ':
                # columns[1] = columns[1][:m.start()] + ' ' + columns[1][m.start():]
                pass

        ''' Because of adding space in above code, the position is messed up, hence redoing it. '''
        columns[3] = []

        for m in re.finditer(columns[2], columns[1]):
            columns[3].append([m.start(), m.end()])

        ''' Tokenizing the words again '''
        columns[1] = columns[1].split(' ')
        columns[2] = columns[2].split(' ')
        ''' New column to specify the aspect term location in the list '''
        columns.append([])
        # columns[5] = []
        ''' for multiple positions of aspect term '''
        for elem in columns[3]:
            if elem[0] == 0: #start index of the 1st occurance of the aspect term
                columns[5].append([0])
            else:
                temp_len = elem[0] # assign the start position of the aspect term
                for j, tokens in enumerate(columns[1]):
                    temp_len -= (len(tokens) + 1) # Counting the words till the position - +1 for blank space
                    if temp_len == 0: # Reached the aspect word
                        columns[5].append([j+1])
                        break
            for k in range(len(columns[2])-1): # if multiple words in the aspect term - tag the following words in the sentence
                columns[5][-1].append(columns[5][-1][-1]+1)

        ''' if not found the aspect term location in the list then do '''
        if len(columns[5]) < 1:
            continue

            # pass

        word_wt_list = [[elem, 0] for elem in columns[1]]
        ''' Assigning weights to every word based on the distance from the aspect term '''
        for j, elem in enumerate(columns[5]):
            for k, word in enumerate(columns[1]):
                if k < elem[0]: # Word left to the aspect term
                    dist = abs(elem[0] - k)
                    if word_wt_list[k][1] < 1/dist:
                        word_wt_list[k][1] = 1/dist
                elif k > elem[-1]: # word right to the aspect term : Aspect term can have multiple words
                    dist = abs(k - elem[-1])
                    if word_wt_list[k][1] < 1/dist:
                        word_wt_list[k][1] = 1/dist
            for aspect_word_loc in elem:
                word_wt_list[aspect_word_loc][1] = 1.5

        ''' For duplicate words - if it's aspect term - add the weights, if not, then take the weight which is greater, make the other weight as zero '''
        for j, word_1 in enumerate(word_wt_list):
            for k, word_2 in enumerate(word_wt_list[j+1:]):
                if word_1[0] == word_2[0]:
                    if word_1[1] == 1.5 and word_2[1] == 1.5:
                        # print(line)
                        # print(word_wt_list)
                        word_1[1] += 1.5
                        word_2[1] = 0
                    elif word_1[1] < word_2[1]:
                        # print(line)
                        # print(word_wt_list)
                        word_1[1] = word_2[1]
                        word_2[1] = 0
                    elif word_1[1] > word_2[1]:
                        # print(line)
                        # print(word_wt_list)
                        word_2[1] = 0
                    elif word_1[1] == word_2[1]:
                        word_2[1] = 0


        ''' For removing duplicate words - they have weight zero now'''
        word_wt_list = [elem for elem in word_wt_list if elem[1] != 0]

        if len(columns[2]) > 1 and len(columns[5]) > 1:
            # print(line)
            # print(columns)
            # print(word_wt_list)
            pass

        if columns[0] == '2911_0':
            # print(line)
            # print(word_wt_list)
            pass
        if i == 100:
            # break
            pass

        '''Removing new line character from columns[4] -  the polarity column'''
        columns[4] = columns[4].rstrip('\n')
        X.append(columns[1])
        X1.append(' '.join(columns[1]))

        Y.append(columns[4])
        sentence_weights.append(word_wt_list)


'''Building vocabulary and counting word occurances'''
X_reduced = reduce(lambda x1, x2: x1 + x2, X)
vocab = list(set(X_reduced))
print(vocab[0])
weight_v = np.zeros_like(vocab, dtype = np.float_)
weight_x = []
pf = open('weightxy_data2.csv', 'w')

pf.write(','.join(vocab))
pf.write(',class_')
pf.write('\n')
for i, sentence in enumerate(sentence_weights):
    for word in sentence:
        v_index = vocab.index(word[0])
        weight_v[v_index] = word[1]

    weight_x.append(weight_v)
    if len(sentence) != weight_v[np.where(weight_v > 0)].shape[0]:
        print(sentence)
        print(len(sentence))
        print(weight_v[np.where(weight_v > 0)])

    ''' code to write the data to file '''
    temp = [str(j) for j in weight_v.tolist()]
    pf.write(','.join(temp))
    pf.write(','+str(Y[i]))
    pf.write('\n')
    weight_v = np.zeros_like(weight_v)

pf.close()
weight_x = np.array(weight_x)

print(weight_x.shape)

f.close()



# weight_X2 = np.reshape(weight_X, (len(Y) ,weight_X.shape[0]))
# print(weight_X2.shape)
# tokens = f.split(',') # columns
# remove punctuations
# table = str.maketrans('', '', string.punctuation)
# print(table)
# tokens = [w.translate(table) for w in tokens]
# # remove tokens that are not alphbetic
# tokens = [word for word in tokens if word.isalpha()]
#
# # remove stop words
# stop_words = set(stopwords.words('english'))
# tokens = [w for w in tokens if not w in stop_words]
# # filter out short tokens
# tokens = [word for word in tokens if len(word) > 1]
# print(tokens)


cashew
(3602, 3612)


In [4]:
import pandas as pd
from sklearn import svm

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report
# from sklearn.metrics import accuracy_score
import pickle
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
print('success')
'hello world'

success


'hello world'

In [5]:
'''Load data set'''
df1 = pd.read_csv("weightxy_data2.csv")
print(df1.shape)
print(df1.iloc[:,:-1].shape)
'hello world'

(3602, 3613)
(3602, 3612)


'hello world'

In [59]:
# X_train, X_test, Y_train, Y_test = train_test_split(df1.iloc[:,:-1], df1['class_'], test_size = 0.3, random_state =42)
X, Y = df1.iloc[:,:-1], df1['class_']
print(Y)

# svm_clf = svm.SVC(kernel = 'rbf')
# svm_clf = svm.LinearSVC(C = 0.1, random_state = 0) #loss = 'hinge') #, multi_class = 'crammer_singer')

# svm_clf.fit(X, Y)
print('success')
print(X.shape)
print(Y.shape)
'hello world'

0      -1
1       1
2       1
3       1
4       0
5       1
6       1
7       1
8       1
9       0
10      0
11      1
12      1
13     -1
14     -1
15     -1
16      0
17     -1
18      0
19      0
20      0
21      0
22      1
23      1
24      1
25      1
26      0
27      1
28     -1
29     -1
       ..
3572    0
3573    1
3574   -1
3575    0
3576   -1
3577    0
3578    0
3579   -1
3580    0
3581    0
3582    1
3583    0
3584   -1
3585    0
3586    0
3587    0
3588    0
3589    0
3590    0
3591    0
3592    1
3593    1
3594    1
3595    1
3596    0
3597    0
3598    0
3599    0
3600    0
3601    0
Name: class_, Length: 3602, dtype: int64
success
(3602, 3612)
(3602,)


'hello world'

In [7]:
y_pred = cross_validate(svm_clf, X, Y, cv = 10, scoring = 'accuracy')

print(y_pred)
print('success')
'hello world'


{'fit_time': array([0.89361811, 0.22660446, 0.21858025, 0.22259307, 0.22660375,
       0.22660565, 0.2215898 , 0.2165761 , 0.2256    , 0.36396813]), 'score_time': array([8.65012240e+00, 4.01091576e-03, 4.01091576e-03, 5.01346588e-03,
       5.01275063e-03, 5.01346588e-03, 5.01465797e-03, 5.01227379e-03,
       4.01043892e-03, 4.01186943e-03]), 'test_score': array([0.72099448, 0.72651934, 0.73480663, 0.66204986, 0.72222222,
       0.70194986, 0.70194986, 0.67688022, 0.70473538, 0.67966574]), 'train_score': array([0.90432099, 0.90617284, 0.90401235, 0.91237272, 0.90746453,
       0.90595128, 0.90718471, 0.91026827, 0.90595128, 0.90595128])}
success


'hello world'

In [8]:
'''Cross validation'''
y_pred = cross_val_predict(svm_clf, X, Y, cv = 10) #, scoring = 'accuracy')

print(y_pred)
print('success')
'hello world'


mod_accracy = accuracy_score(y_pred, Y)
print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

target_names = ['0', '1', '-1']
print('Pr, Re scores wrt each class')
print(classification_report(y_pred, Y, target_names=target_names))


[-1 -1  1 ...  1  0  0]
success


Model Accuracy is: 70.3220%
Pr, Re scores wrt each class
             precision    recall  f1-score   support

          0       0.49      0.60      0.54       653
          1       0.31      0.52      0.39       384
         -1       0.90      0.76      0.82      2565

avg / total       0.76      0.70      0.72      3602



In [ ]:
print(scores)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaBoost_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2)) #, n_estimators=2, learning_rate=1)
scores = cross_validate(adaBoost_clf, X, Y, cv = 10)
print(scores.keys())
print('success')
'hello world'

print(scores)

y_pred = cross_val_predict(adaBoost_clf, X, Y, cv = 10) #, scoring = 'accuracy')

print(y_pred)
print('success')
'hello world'


mod_accracy = accuracy_score(y_pred, Y)
print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

target_names = ['0', '1', '-1']
print('Pr, Re scores wrt each class')
print(classification_report(y_pred, Y, target_names=target_names))


In [ ]:
# Using MNB: 
from sklearn.naive_bayes import MultinomialNB
mnb_clf = MultinomialNB()

scores = cross_validate(mnb_clf, X, Y, cv = 10)
print(scores.keys())
print('success')
'hello world'

print(scores)

y_pred = cross_val_predict(mnb_clf, X, Y, cv = 10) #, scoring = 'accuracy')

print(y_pred)
print('success')
'hello world'


mod_accracy = accuracy_score(y_pred, Y)
print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

target_names = ['0', '1', '-1']
print('Pr, Re scores wrt each class')
print(classification_report(y_pred, Y, target_names=target_names))



In [18]:
from sklearn.ensemble import RandomForestClassifier
randomForest_clf = RandomForestClassifier()# (max_depth=2, random_state=4)

scores = cross_validate(randomForest_clf, X, Y, cv = 10)
print(scores.keys())
print('success')
'hello world'

print(scores)

y_pred = cross_val_predict(randomForest_clf, X, Y, cv = 10) #, scoring = 'accuracy')

print(y_pred)
print('success')
'hello world'


mod_accracy = accuracy_score(y_pred, Y)
print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

target_names = ['0', '1', '-1']
print('Pr, Re scores wrt each class')
print(classification_report(y_pred, Y, target_names=target_names))


dict_keys(['fit_time', 'score_time', 'test_score', 'train_score'])
success
{'fit_time': array([1.34776497, 0.71089029, 0.70386958, 0.99575043, 0.69885802,
       0.69484711, 0.79618025, 0.71137023, 1.37406015, 0.71088815]), 'score_time': array([0.01292443, 0.07402015, 0.00802159, 0.00701857, 0.00802112,
       0.00801992, 0.00902414, 0.00902438, 0.00902414, 0.00802135]), 'test_score': array([0.68508287, 0.63812155, 0.69889503, 0.65927978, 0.68333333,
       0.6545961 , 0.6183844 , 0.65181058, 0.67130919, 0.64345404]), 'train_score': array([0.98611111, 0.98611111, 0.98611111, 0.9861154 , 0.98673658,
       0.98674067, 0.98828245, 0.98427382, 0.9879741 , 0.98519889])}
[-1  1  1 ...  1  1 -1]
success


Model Accuracy is: 65.1860%
Pr, Re scores wrt each class
             precision    recall  f1-score   support

          0       0.40      0.52      0.45       620
          1       0.24      0.40      0.30       376
         -1       0.87      0.72      0.78      2606

avg / total       0.

In [9]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(30,30,30))
X, Y = df1.iloc[:,:-1], df1['class_']
mlp.fit(X, Y)

y_pred = cross_val_predict(mlp, X, Y, cv = 10)
print(y_pred)
print('success')
'hello world'


mod_accracy = accuracy_score(y_pred, Y)
print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

target_names = ['0', '1', '-1']
print('Pr, Re scores wrt each class')
print(classification_report(y_pred, Y, target_names=target_names))


[-1  1  1 ...  1  1  0]
success


Model Accuracy is: 65.8801%
Pr, Re scores wrt each class
             precision    recall  f1-score   support

          0       0.49      0.53      0.51       749
          1       0.38      0.41      0.39       590
         -1       0.80      0.77      0.79      2263

avg / total       0.67      0.66      0.66      3602



In [48]:
import tensorflow as tf
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
import numpy

# neurons = 4
# batch_size = 1
# layer = LSTM(units = 4, stateful=True)
# print(layer)

numpy.random.seed(7)
n_batch = 1
n_epoch = 1000
n_neurons = 10

model = Sequential()




# model.add(Dense(1))

model.add(Dense(12, input_dim=3612, activation='relu'))
# model.add(LSTM(2, stateful=True, batch_size = X.shape[0]))
model.add(Dense(3612, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(Y, num_classes=3)
# Fit the model
'''The ***batch size*** limits the number of samples to be shown to the network before a weight update can be performed. 
This same limitation is then imposed when making predictions with the fit model.
Specifically, the batch size used when fitting your model controls how many predictions you must make at a time.'''

model.fit(X, one_hot_labels, epochs=150, batch_size=10)


# y_pred = cross_val_predict(model, X, Y, cv = 10) #, scoring = 'accuracy')


# evaluate the model
y_pred = model.predict(X)
print(y_pred)
scores = model.evaluate(X, Y)
print(scores)
print(model.metrics_names[1])


print("\n%s: %.4f%%" % (model.metrics_names[1], scores[1]*100))


ValueError: Error when checking target: expected dense_38 to have shape (1,) but got array with shape (3,)

In [41]:
y_pred = np.array(y_pred)
for label_index, label in enumerate(y_pred):
    if label == '-1.':
        print(True)
    y_pred[label_index] = int(label)
    print(y_pred[label_index])



# print(X.shape)
# print(Y.shape)

# target_names = ['0', '1', '-1']
# print('Pr, Re scores wrt each class')
# print(classification_report(y_pred, Y, target_names=target_names))


C:\Users\Shubadra\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]


[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[1.]
[0.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[0.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[0.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[0.]
[0.]
[1.]
[0.]
[0.]
[0.]
[0.]
[0.]
[1.]
[1.]
[1.]
[0.]
[0.]
[0.]
[0.]


In [47]:
import keras
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))
print(labels)
# Convert labels to categorical one-hot encoding
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=10, batch_size=32)

[[3]
 [7]
 [1]
 [2]
 [5]
 [3]
 [9]
 [5]
 [6]
 [8]
 [5]
 [3]
 [5]
 [9]
 [0]
 [0]
 [0]
 [0]
 [7]
 [1]
 [3]
 [4]
 [0]
 [4]
 [0]
 [0]
 [9]
 [0]
 [7]
 [2]
 [7]
 [2]
 [5]
 [7]
 [7]
 [5]
 [8]
 [1]
 [6]
 [5]
 [0]
 [0]
 [8]
 [3]
 [2]
 [2]
 [9]
 [9]
 [5]
 [5]
 [9]
 [9]
 [8]
 [8]
 [3]
 [0]
 [8]
 [0]
 [8]
 [5]
 [7]
 [7]
 [4]
 [0]
 [4]
 [6]
 [7]
 [3]
 [9]
 [0]
 [1]
 [6]
 [2]
 [5]
 [8]
 [1]
 [8]
 [6]
 [0]
 [5]
 [0]
 [3]
 [2]
 [7]
 [5]
 [6]
 [7]
 [3]
 [3]
 [8]
 [3]
 [7]
 [9]
 [0]
 [4]
 [6]
 [0]
 [1]
 [6]
 [0]
 [1]
 [8]
 [3]
 [0]
 [9]
 [7]
 [9]
 [9]
 [0]
 [6]
 [6]
 [3]
 [2]
 [4]
 [9]
 [9]
 [5]
 [8]
 [1]
 [0]
 [5]
 [2]
 [2]
 [9]
 [0]
 [4]
 [1]
 [9]
 [8]
 [8]
 [2]
 [0]
 [4]
 [6]
 [0]
 [9]
 [5]
 [9]
 [7]
 [6]
 [3]
 [8]
 [7]
 [8]
 [6]
 [4]
 [5]
 [2]
 [7]
 [0]
 [2]
 [0]
 [2]
 [9]
 [4]
 [7]
 [0]
 [7]
 [8]
 [6]
 [4]
 [6]
 [0]
 [4]
 [5]
 [9]
 [9]
 [2]
 [2]
 [4]
 [0]
 [3]
 [4]
 [5]
 [0]
 [2]
 [0]
 [9]
 [8]
 [2]
 [1]
 [8]
 [6]
 [5]
 [3]
 [0]
 [6]
 [4]
 [5]
 [0]
 [0]
 [1]
 [1]
 [0]
 [2]
 [4]
 [7]
 [5]
 [8]
 [3]


In [67]:
##
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

X, Y = df1.iloc[:,:-1], df1['class_']

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)




# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=3612, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     print(model.predict(X))
    
    return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=5, verbose=2)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


# scores = cross_validate(estimator, X, dummy_y, cv = 10)
# print(scores.keys())
print('success')
'hello world'

print(results)

# y_pred = cross_val_predict(randomForest_clf, X, Y, cv = 10) #, scoring = 'accuracy')

# print(y_pred)
print('success')
'hello world'

# y_pred = estimator.predict(X)
# print(y_pred)
# scores = estimator.evaluate(X, dummy_y)

scores = estimator.predict_classes(X, verbose = 2)
print(scores)
print(model.metrics_names[1])


print("\n%s: %.4f%%" % (estimator.metrics_names[1], scores[1]*100))



# mod_accracy = accuracy_score(y_pred, Y)
# print('\n\nModel Accuracy is: {:.4%}' .format(mod_accracy))

# target_names = ['0', '1', '-1']
# print('Pr, Re scores wrt each class')
# print(classification_report(y_pred, Y, target_names=target_names))



Baseline: 75.71% (2.27%)
success
[0.7396122  0.7562327  0.77777779 0.75555557 0.71666668 0.7888889
 0.77500001 0.72777779 0.75277779 0.78055557]
success


AttributeError: 'KerasClassifier' object has no attribute 'model'